# Messari Tokens Alert

In this notebook, we explore [Messari's API](https://messari.io/account/api) so we can build on top of it.

💡 Please add the API key in `.env`.

### Install dependencies

In [221]:
import os
import requests
import messari
import numpy as np
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
from urllib.parse import urljoin
from messari.messari import Messari

In [222]:
load_dotenv()
env_path = Path('.')/'.env'
load_dotenv(dotenv_path=env_path)
 
MESSARI_API = os.getenv("MESSARI_API")
MESSARI_URL = os.getenv("MESSARI_URL")
TOKENS_LIST = os.getenv("TOKENS_LIST")

if not (bool(MESSARI_API) and bool(MESSARI_URL) and bool(TOKENS_LIST)):
    raise Exception('Please add config to .env file')

In [223]:
# How many assets should be retrieved
ASSETS_LIMIT = 100

messari = Messari(MESSARI_API)

### Define util methods

In [224]:
def send_request(endpoint, less_log=False) -> dict:
    
    url = urljoin(MESSARI_URL, endpoint)
    
    if not less_log:
        print('Sending request to: {}'.format(url))
    
    try:
        r = requests.get(url)
        
    except requests.exceptions.HTTPError  as e:
        raise Exception('Error querying to {0}: {1}'.format(url, e.response.text))
    
    if r.status_code == 200:
        return r.json()
    else:
        raise Exception('Query failed - return code:{}.'.format(r.status_code))

In [225]:
def handle_KeyError(response, key) -> str:    
    try:
        return response[key]
    except KeyError as e:
         raise Exception('Response cannot be parsed.'.format(e))

In [235]:
def open_file(filename) -> list:
    try:
        with open(filename,'r') as f:
            return f.readlines()
    except IOError as e:
         raise Exception('Could not open {0}: {1}'.format(filename, e))

### Define API methods

In [226]:
def parse_assets_list(response):
    
    assets = pd.DataFrame(handle_KeyError(response, 'data'))
    assets["timestamp"] = pd.to_datetime(handle_KeyError(handle_KeyError(response, 'status'), 'timestamp'))    

    del assets["id"]
    del assets["serial_id"]
    del assets["contract_addresses"]
    del assets["_internal_temp_agora_id"]
    
    return assets

In [227]:
def get_assets():
    
    endpoint = '/api/v2/assets?fields&limit={}'.format(ASSETS_LIMIT)
    response = send_request(endpoint)

    return parse_assets_list(response)

In [228]:
def parse_asset_metrics(response):

    info = {}
    info['name'] = response['name']
    info['symbol'] = response['symbol']
    info['marketcap'] = response['marketcap']
    info['supply'] = response['supply']
    info['market_data'] = response['market_data']
    
    '''
    other data: 
    blockchain_stats_24_hours
    market_data_liquidity
    all_time_high
    token_sale_stats
    roi_data
    on_chain_data
    exchange_flows
    supply_activity
    '''
    
    return info

In [229]:
def get_asset_metrics(asset):
    
    endpoint = "https://data.messari.io/api/v1/assets/{}/metrics".format(asset)
    response = requests.get(endpoint).json()
    
    profile = handle_KeyError(response, 'data')
    
    del profile["id"]
    del profile["serial_id"]
    del profile["_internal_temp_agora_id"]
    del profile["contract_addresses"]
    del profile["alert_messages"]
    del profile["market_data"]['ohlcv_last_1_hour']
    del profile["developer_activity"]
    del profile["mining_stats"]
    del profile["misc_data"]
    del profile["reddit"]
    del profile["miner_flows"]

    return parse_asset_metrics(response['data'])

### Let's get a list of assets

In [230]:
assets_list = get_assets()
assets_list

Sending request to: https://data.messari.io/api/v2/assets?fields&limit=100


,symbol,name,slug,metrics,profile,timestamp
0,ACU,Acuity,acuity,"{'market_data': {'price_usd': None, 'price_btc...","{'general': {'overview': {'is_verified': None,...",2022-02-22 22:00:31.499335651+00:00
1,None,Rigoblock,rigoblock,"{'market_data': {'price_usd': None, 'price_btc...","{'general': {'overview': {'is_verified': None,...",2022-02-22 22:00:31.499335651+00:00
2,DAXX,DaxxCoin,daxxcoin,"{'market_data': {'price_usd': None, 'price_btc...","{'general': {'overview': {'is_verified': None,...",2022-02-22 22:00:31.499335651+00:00
3,POS,PokemonSpace,pokemonspace,"{'market_data': {'price_usd': None, 'price_btc...","{'general': {'overview': {'is_verified': None,...",2022-02-22 22:00:31.499335651+00:00
4,None,AgentMile,agentmile,"{'market_data': {'price_usd': None, 'price_btc...","{'general': {'overview': {'is_verified': None,...",2022-02-22 22:00:31.499335651+00:00
...,...,...,...,...,...,...
95,DICE,BetDice,betdice,"{'market_data': {'price_usd': None, 'price_btc...","{'general': {'overview': {'is_verified': None,...",2022-02-22 22:00:31.499335651+00:00
96,TKY,THEKEY,thekey-1,"{'market_data': {'price_usd': None, 'price_btc...","{'general': {'overview': {'is_verified': None,...",2022-02-22 22:00:31.499335651+00:00
97,DBALL,DrakeBall Token [OLD],drakeball-token,"{'market_data': {'price_usd': None, 'price_btc...","{'general': {'overview': {'is_verified': None,...",2022-02-22 22:00:31.499335651+00:00
98,BSDS,Basis Dollar Share,basis-dollar-share,"{'market_data': {'price_usd': None, 'price_btc...","{'general': {'overview': {'is_verified': None,...",2022-02-22 22:00:31.499335651+00:00


At any time we can look at one asset with:

In [231]:
assets_list.at[1, 'metrics']

{'market_data': {'price_usd': None,
  'price_btc': None,
  'price_eth': None,
  'volume_last_24_hours': None,
  'real_volume_last_24_hours': None,
  'volume_last_24_hours_overstatement_multiple': None,
  'percent_change_usd_last_1_hour': None,
  'percent_change_btc_last_1_hour': None,
  'percent_change_eth_last_1_hour': None,
  'percent_change_usd_last_24_hours': None,
  'percent_change_btc_last_24_hours': None,
  'percent_change_eth_last_24_hours': None,
  'ohlcv_last_1_hour': None,
  'ohlcv_last_24_hour': None,
  'last_trade_at': None},
 'marketcap': {'rank': None,
  'marketcap_dominance_percent': None,
  'current_marketcap_usd': None,
  'y_2050_marketcap_usd': None,
  'y_plus10_marketcap_usd': None,
  'liquid_marketcap_usd': None,
  'volume_turnover_last_24_hours_percent': None,
  'realized_marketcap_usd': None,
  'outstanding_marketcap_usd': None},
 'supply': {'y_2050': None,
  'y_plus10': None,
  'liquid': None,
  'circulating': None,
  'y_2050_issued_percent': None,
  'annual_inf

### Let's get an asset metric

In [232]:
asset = 'btc'

get_asset_metrics(asset)

{'name': 'Bitcoin',
 'symbol': 'BTC',
 'marketcap': {'rank': 1,
  'marketcap_dominance_percent': 44.69924563469265,
  'current_marketcap_usd': 718562503705.6885,
  'y_2050_marketcap_usd': 795158363565.1091,
  'y_plus10_marketcap_usd': 783373136584.0762,
  'liquid_marketcap_usd': 718865443739.3606,
  'volume_turnover_last_24_hours_percent': 0.849881607220666,
  'realized_marketcap_usd': 457041344492.9346,
  'outstanding_marketcap_usd': 703550978541.0667},
 'supply': {'y_2050': 20986335.65,
  'y_plus10': 20675292.29,
  'liquid': 18972763.39,
  'circulating': 18964768,
  'y_2050_issued_percent': 90.40531756671871,
  'annual_inflation_percent': 1.7717724776833328,
  'stock_to_flow': 56.44065547894402,
  'y_plus10_issued_percent': 91.76539380377487,
  'supply_revived_90d': 18964642.18175567},
 'market_data': {'price_usd': 37874.19261866894,
  'price_btc': 1,
  'price_eth': 14.540953760649478,
  'volume_last_24_hours': 34493126741971.27,
  'real_volume_last_24_hours': 5826023948.133953,
  'v

### Let's print it cool

In [253]:
def print_market_data_asset(data):
    
    marketcap = data['market_data']
    
    print('{}:'.format(data['name']))
    print('    Price USD: {}'.format(marketcap['price_usd']))
    print('    Price BTC: {}'.format(marketcap['price_btc']))
    print('    Price ETH: {}'.format(marketcap['price_eth']))
    print('    Volume last 24h: {}'.format(marketcap['volume_last_24_hours']))
    print('    Real volume last 24h: {}'.format(marketcap['real_volume_last_24_hours']))
    print('    Percent change USD last 24h: {}'.format(marketcap['percent_change_usd_last_24_hours']))
    print()

In [254]:
def print_market_data():
    
    print('🚨 MARKET DATA FOR ASSETS') 
    print()
    print()
    
    assets = open_file(TOKENS_LIST)
    
    for asset in assets:
        print_market_data_asset(get_asset_metrics(asset.strip()))

In [256]:
print_market_data()

🚨 MARKET DATA FOR ASSETS


Bitcoin:
    Price USD: 37967.99431986777
    Price BTC: 1
    Price ETH: 14.543348565090518
    Volume last 24h: 34662988777422.668
    Real volume last 24h: 5848619785.708213
    Percent change USD last 24h: 2.2403363747124856

Ethereum:
    Price USD: 2610.69232181529
    Price BTC: 0.06876015055044946
    Price ETH: 1
    Volume last 24h: 11070369063.56649
    Real volume last 24h: 4989968077.851881
    Percent change USD last 24h: 0.7866751750787413

SushiSwap:
    Price USD: 3.338417348379207
    Price BTC: 8.792737384751002e-05
    Price ETH: 0.001278754010110685
    Volume last 24h: 55686963.82117409
    Real volume last 24h: 52978936.24277977
    Percent change USD last 24h: -1.2826316179337691

Alchemix:
    Price USD: 132.99232203455696
    Price BTC: 0.0035034514878862157
    Price ETH: 0.05086320554676411
    Volume last 24h: 0
    Real volume last 24h: 0
    Percent change USD last 24h: -3.1803346102124443

Index Cooperative:
    Price USD: 5.78